In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display

import cv2 
import json
import time
import math
import sympy as sp

In [2]:
# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Maps bones to a matplotlib color name.
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
  """Returns high confidence keypoints and edges for visualization.

  Args:
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
    keypoint_threshold: minimum confidence score for a keypoint to be
      visualized.

  Returns:
    A (keypoints_xy, edges_xy, edge_colors) containing:
      * the coordinates of all keypoints of all detected entities;
      * the coordinates of all skeleton edges of all detected entities;
      * the colors in which the edges should be plotted.
  """
  keypoints_all = []
  keypoint_edges_all = []
  edge_colors = []
  num_instances, _, _, _ = keypoints_with_scores.shape
  for idx in range(num_instances):
    kpts_x = keypoints_with_scores[0, idx, :, 1]
    kpts_y = keypoints_with_scores[0, idx, :, 0]
    kpts_scores = keypoints_with_scores[0, idx, :, 2]
    kpts_absolute_xy = np.stack(
        [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[
        kpts_scores > keypoint_threshold, :]
    keypoints_all.append(kpts_above_thresh_absolute)

    for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
      if (kpts_scores[edge_pair[0]] > keypoint_threshold and
          kpts_scores[edge_pair[1]] > keypoint_threshold):
        x_start = kpts_absolute_xy[edge_pair[0], 0]
        y_start = kpts_absolute_xy[edge_pair[0], 1]
        x_end = kpts_absolute_xy[edge_pair[1], 0]
        y_end = kpts_absolute_xy[edge_pair[1], 1]
        line_seg = np.array([[x_start, y_start], [x_end, y_end]])
        keypoint_edges_all.append(line_seg)
        edge_colors.append(color)
  if keypoints_all:
    keypoints_xy = np.concatenate(keypoints_all, axis=0)
  else:
    keypoints_xy = np.zeros((0, 17, 2))

  if keypoint_edges_all:
    edges_xy = np.stack(keypoint_edges_all, axis=0)
  else:
    edges_xy = np.zeros((0, 2, 2))
  return keypoints_xy, edges_xy, edge_colors


def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot

def to_gif(images, duration):
  """Converts image sequence (4D numpy array) to gif."""
  imageio.mimsave('./animation.gif', images, duration=duration)
  return embed.embed_file('./animation.gif')

def progress(value, max=100):
  return HTML("""
      <progress
          value='{value}'
          max='{max}',
          style='width: 100%'
      >
          {value}
      </progress>
  """.format(value=value, max=max))

In [3]:
model_name = "movenet_lightning"

if "movenet_lightning" in model_name:
  module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/2")
  input_size = 192
elif "movenet_thunder" in model_name:
  module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/2")
  input_size = 256
else:
  raise ValueError("Unsupported model name: %s" % model_name)

def movenet(input_image):
  """Runs detection on an input image.

  Args:
    input_image: A [1, height, width, 3] tensor represents the input image
      pixels. Note that the height/width should already be resized and match the
      expected input resolution of the model before passing into this function.

  Returns:
    A [1, 1, 17, 3] float numpy array representing the predicted keypoint
    coordinates and scores.
  """
  model = module.signatures['serving_default']

  # SavedModel format expects tensor type of int32.
  input_image = tf.cast(input_image, dtype=tf.int32)
  # Run model inference.
  outputs = model(input_image)
  # Output is a [1, 1, 17, 3] tensor.
  keypoints_with_scores = outputs['output_0'].numpy()
  return keypoints_with_scores

In [4]:
#angle calculations
#find angle
# Python3 program for the above approach
from math import atan
# from angle_control_func import move_joint
# Function to find the
# angle between two lines
def findAngle(point1, point2, point3):
    M1 = (point1[1]-point2[1])/(point1[0]-point2[0])
    M2 = (point3[1]-point2[1])/(point3[0]-point2[0])
    PI = 3.14159265
	
	# Store the tan value of the angle
    angle = abs((M2 - M1) / (1 + M1 * M2))

	# Calculate tan inverse of the angle
    ret = atan(angle)

	# Print the result
    return (round(ret, 2))

In [5]:
a = -1
b = -1

def findElbow3D(shoulder, elbow, wrist):
    wtoe = sp.sqrt(sp.Pow(wrist[0] - elbow[0], 2) + sp.Pow(wrist[1] - elbow[1], 2))
    etos = sp.sqrt(sp.Pow(shoulder[0] - elbow[0], 2) + sp.Pow(shoulder[1] - elbow[1], 2))
    a = max(a , etos)
    b = max(b , wtoe)

    shoulder[2] = 0
    elbow[2] = sp.sqrt(sp.Pow(a, 2) - sp.Pow(shoulder[0] - elbow[0], 2) - sp.Pow(shoulder[1] - elbow[1], 2))
    wrist[2] = elbow[2] + sp.sqrt(sp.Pow(b, 2) - sp.Pow(wrist[0] - elbow[0], 2) - sp.Pow(wrist[1] - elbow[1], 2))
    
    d = sp.sqrt(sp.Pow(wrist[0] - shoulder[0], 2) + sp.Pow(shoulder[1] - wrist[1], 2) + sp.Pow(shoulder[2] - wrist[2], 2))
    x, y = sp.symbols('x y')
    equation1 = sp.Eq(a * x + b * y, d)
    equation2 = sp.Eq(sp.Pow(a, 2) * (x**2) - sp.Pow(b, 2) * (y**2), sp.Pow(a, 2) - sp.Pow(b, 2))
    
    solution = sp.solve((equation1, equation2), (x, y))
    alpha = math.asin(solution[0][0])
    beta = math.asin(solution[0][1])

    return round(alpha,2) , round(beta,2)


In [6]:
# def calculate_head_yaw(left_eye, right_eye, left_ear, right_ear):
#     # Calculate midpoints
#     midpoint_eyes = ((left_eye[0] + right_eye[0]) / 2, (left_eye[1] + right_eye[1]) / 2)
#     midpoint_ears = ((left_ear[0] + right_ear[0]) / 2, (left_ear[1] + right_ear[1]) / 2)

#     # Calculate angle to horizontal
#     delta_x = midpoint_ears[0] - midpoint_eyes[0]
#     delta_y = midpoint_ears[1] - midpoint_eyes[1]

#     # Calculate angle in radians
#     angle = math.atan2(delta_y, delta_x)

#     return angle


def calculate_head_yaw(left_eye, right_eye, left_ear, right_ear, left_shoulder, right_shoulder):
    # Calculate midpoints
    midpoint_eyes = ((left_eye[0] + right_eye[0]) / 2, (left_eye[1] + right_eye[1]) / 2)
    midpoint_ears = ((left_ear[0] + right_ear[0]) / 2, (left_ear[1] + right_ear[1]) / 2)
    midpoint_shoulders = ((left_shoulder[0] + right_shoulder[0]) / 2, (left_shoulder[1] + right_shoulder[1]) / 2)

    # Adjust the midpoints based on shoulder orientation
    adjusted_midpoint_eyes = (midpoint_eyes[0] - midpoint_shoulders[0], midpoint_eyes[1] - midpoint_shoulders[1])
    adjusted_midpoint_ears = (midpoint_ears[0] - midpoint_shoulders[0], midpoint_ears[1] - midpoint_shoulders[1])

    # Calculate angle to horizontal
    delta_x = adjusted_midpoint_ears[0] - adjusted_midpoint_eyes[0]
    delta_y = adjusted_midpoint_ears[1] - adjusted_midpoint_eyes[1]

    # Calculate angle in radians
    angle = math.atan2(delta_y, delta_x)

    return angle



In [7]:
def findKnee(Hip, foot):
    l = math.sqrt((Hip[0]-foot[0])**2 + (Hip[1]-foot[1])**2)
    b = 0.10882269
    c = 0.10681073

    cos_ang = ((c**2)+(b**2)-(l**2))/(2*b*c)
    print(cos_ang)
    knee_angle = math.acos(abs(cos_ang))
    return knee_angle

In [8]:
# def calculate_angle(p1, p2, p3):
#     """
#     Calculate the angle formed by three points: p1, p2, and p3.
#     p2 is the vertex point where the angle is formed.
#     """
#     # Create vectors
#     vector1 = (p1[0] - p2[0], p1[1] - p2[1])
#     vector2 = (p3[0] - p2[0], p3[1] - p2[1])

#     # Calculate dot product
#     dot_product = vector1[0]*vector2[0] + vector1[1]*vector2[1]

#     # Calculate magnitudes
#     magnitude1 = math.sqrt(vector1[0]**2 + vector1[1]**2)
#     magnitude2 = math.sqrt(vector2[0]**2 + vector2[1]**2)

#     # Calculate the angle in radians
#     angle = math.acos(dot_product / (magnitude1 * magnitude2))

#     return angle

# def knee_pitch(hip,knee,ankle):
#     """
#     Calculate the knee pitch angles for both legs.
#     Coordinates are given as (x, y) tuples.
#     """
#     knee_pitch = calculate_angle(hip, knee, ankle)

#     return knee_pitch

# def knee_pitch(left_hip, left_knee, left_ankle):
#     # Helper function to calculate the Euclidean distance between two points
#     def distance(p1, p2):
#         return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
    
#     # Calculate the lengths of the sides of the triangle
#     a = distance(left_hip, left_knee)  # hip to knee
#     b = distance(left_knee, left_ankle)  # knee to ankle
#     c = distance(left_hip, left_ankle)  # hip to ankle

#     # Calculate the knee pitch using the law of cosines
#     # Ensure the denominator is not zero to avoid division by zero
#     if a * b != 0:
#         cosine_angle = (a**2 + b**2 - c**2) / (2 * a * b)
#         # Ensure the cosine value is in the valid range [-1, 1] to avoid math domain error
#         cosine_angle = max(min(cosine_angle, 1), -1)
#         knee_pitch = math.acos(cosine_angle)
#     else:
#         knee_pitch = 0  # In case the points are overlapping

#     return knee_pitch

# def calculate_angles(left_hip, right_hip, left_knee, right_knee, left_ankle, right_ankle):
#     # Helper function to calculate angle between three points
#     def angle_between_points(p1, p2, p3):
#         a = math.sqrt((p2[0] - p3[0])**2 + (p2[1] - p3[1])**2)
#         b = math.sqrt((p1[0] - p3[0])**2 + (p1[1] - p3[1])**2)
#         c = math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
#         return math.acos((a**2 + c**2 - b**2) / (2 * a * c))

#     # Average the coordinates for left and right joints to get a single value for each joint
#     hip = [(left_hip[0] + right_hip[0]) / 2, (left_hip[1] + right_hip[1]) / 2]
#     knee = [(left_knee[0] + right_knee[0]) / 2, (left_knee[1] + right_knee[1]) / 2]
#     ankle = [(left_ankle[0] + right_ankle[0]) / 2, (left_ankle[1] + right_ankle[1]) / 2]

#     # Calculate the angles
#     knee_pitch = angle_between_points(hip, knee, ankle)
#     hip_pitch = angle_between_points([hip[0], hip[1] - 1], hip, knee)
#     ankle_pitch = angle_between_points(knee, ankle, [ankle[0] + 1, ankle[1]])

#     return knee_pitch, hip_pitch, ankle_pitch

def calculate_angle(p1, p2, p3):
    """
    Helper function to calculate the angle formed by three points.
    :param p1, p2, p3: tuples representing the points (x, y).
    :return: angle in radians.
    """
    a = (p2[0] - p1[0], p2[1] - p1[1])  # vector from p1 to p2
    b = (p3[0] - p2[0], p3[1] - p2[1])  # vector from p2 to p3

    dot_product = a[0] * b[0] + a[1] * b[1]
    magnitude_a = math.sqrt(a[0]**2 + a[1]**2)
    magnitude_b = math.sqrt(b[0]**2 + b[1]**2)

    # Angle in radians
    angle = math.acos(dot_product / (magnitude_a * magnitude_b))

    # Adjusting angle based on your requirements
    if a[1] < 0:  # Leg/hip goes backward
        angle = -angle

    return angle

def knee_hip_pitch(left_shoulder, right_shoulder, left_hip, right_hip, left_knee, right_knee, left_ankle, right_ankle):
    """
    Calculate the knee pitch and hip pitch angles.
    :param left_shoulder, right_shoulder, left_hip, right_hip, left_knee, right_knee, left_ankle, right_ankle: coordinates (x, y) of each joint.
    :return: knee_pitch, hip_pitch in radians.
    """

    # Average hip and knee coordinates for symmetry
    hip = ((left_hip[0] + right_hip[0]) / 2, (left_hip[1] + right_hip[1]) / 2)
    knee = ((left_knee[0] + right_knee[0]) / 2, (left_knee[1] + right_knee[1]) / 2)
    ankle = ((left_ankle[0] + right_ankle[0]) / 2, (left_ankle[1] + right_ankle[1]) / 2)

    # Assuming shoulders are aligned horizontally
    shoulder = ((left_shoulder[0] + right_shoulder[0]) / 2, (left_shoulder[1] + right_shoulder[1]) / 2)

    # Calculate knee pitch
    knee_pitch = calculate_angle(hip, knee, ankle)

    # Calculate hip pitch
    hip_pitch = calculate_angle(shoulder, hip, knee)

    return knee_pitch, hip_pitch

In [9]:
def shoulder_yaw_angle(shoulder, elbow, wrist):
    # Consider only the x-coordinates for a front view
    shoulder_x, _,a = shoulder
    elbow_x, _,b = elbow
    wrist_x, _,c = wrist

    # Calculate the vectors representing the arm segments
    shoulder_to_elbow_vector = elbow_x - shoulder_x
    elbow_to_wrist_vector = wrist_x - elbow_x

    # Calculate the magnitudes of the vectors
    magnitude_shoulder_to_elbow = abs(shoulder_to_elbow_vector)
    magnitude_elbow_to_wrist = abs(elbow_to_wrist_vector)

    # Calculate the cosine of the angle between the vectors
    cos_angle = 0.0  # Default to 0 to handle division by zero

    if magnitude_shoulder_to_elbow != 0 and magnitude_elbow_to_wrist != 0:
        cos_angle = shoulder_to_elbow_vector / (magnitude_shoulder_to_elbow * magnitude_elbow_to_wrist)

    # Calculate the shoulder yaw angle in radians
    shoulder_yaw_angle_rad = math.acos(abs(cos_angle))

    return shoulder_yaw_angle_rad

In [10]:
def arm_pitch(wrist, elbow, shoulder, hip):
    pitch_angle = math.atan2(wrist[1] - shoulder[1], wrist[0] - shoulder[0]) - math.atan2(hip[1] - shoulder[1], hip[0] - shoulder[0])
    return pitch_angle

In [12]:
vid = cv2.VideoCapture(0) 
  
while(True):
    ret, frame = vid.read() 

    image_data = frame
    
    image = tf.convert_to_tensor(image_data, dtype=tf.uint8)
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    keypoints_with_scores = movenet(input_image)

    Lshoulder = keypoints_with_scores[0][0][5]
    Lelbow = keypoints_with_scores[0][0][7]
    Lwrist = keypoints_with_scores[0][0][9]
    Lhip = keypoints_with_scores[0][0][11]
    LAnkle = keypoints_with_scores[0][0][15]
    LeftEar = keypoints_with_scores[0][0][3]
    LKnee = keypoints_with_scores[0][0][13]
    LeftEye = keypoints_with_scores[0][0][1]

    RightEar = keypoints_with_scores[0][0][4]
    Rshoulder = keypoints_with_scores[0][0][6]
    Relbow = keypoints_with_scores[0][0][10]
    Rwrist = keypoints_with_scores[0][0][9]
    Rhip = keypoints_with_scores[0][0][12]
    RAnkle = keypoints_with_scores[0][0][16]
    RKnee = keypoints_with_scores[0][0][14]
    RightEye = keypoints_with_scores[0][0][2]

    head_angle = calculate_head_yaw(LeftEye , RightEye, LeftEar , RightEar, Lshoulder, Rshoulder)
    Lshoulder_angle = findAngle(Lelbow, Lshoulder, Lhip)
    LElbow_angle= findAngle(Lshoulder , Lelbow , Lwrist)
    Rshoulder_angle = findAngle(Relbow, Rshoulder, Rhip)
    RElbow_angle = findAngle(Rshoulder , Relbow , Rwrist)

    KneePitch , HipPitch = knee_hip_pitch(Lshoulder, Rshoulder, Lhip, Rhip, LKnee, RKnee, LAnkle, RAnkle)

    Lshoulder_pitch = arm_pitch(Lwrist , Lelbow ,Lshoulder, Lhip )
    Rshoulder_pitch = arm_pitch(Rwrist , Relbow ,Rshoulder, Rhip)
    # LShoulder_pitch_angle = shoulder_yaw_angle(Lshoulder , Lelbow , Lwrist)
    # RShoulder_pitch_angle = shoulder_yaw_angle(Rshoulder , Relbow , Rwrist)

    data = {
            "LShoulderRoll": Lshoulder_angle , 
            "RShoulderRoll": -Rshoulder_angle , 
            "LElbowRoll": -LElbow_angle , 
            "RElbowRoll": -RElbow_angle, 
            "HeadYaw": -head_angle , 
            # "LShoulderPitch": -Lshoulder_pitch+1.57,
            # "RShoulderPitch": -Rshoulder_pitch+1.57,
            # "RKneePitch": -KneePitch , 
            # "LKneePitch": -KneePitch , 
            # "LHipPitch": -HipPitch , 
            # "RHipPitch": -HipPitch , 
            # "right_ankle": AnklePitch , 
            # "left_ankle": AnklePitch,
            # "left_shoulder_pitch":LShoulder_pitch_angle,
            # "right_shoulder_pitch":RShoulder_pitch_angle
            }

    with open('angles.json', 'w') as json_file:   
        json.dump(data, json_file)
    time.sleep(1)

    display_image = tf.expand_dims(image, axis=0)
    display_image = tf.cast(tf.image.resize_with_pad(
        display_image, 1280, 1280), dtype=tf.int32)
    output_overlay = draw_prediction_on_image(
        np.squeeze(display_image.numpy(), axis=0), keypoints_with_scores)
    cv2.imshow('output_overlay' ,output_overlay)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
vid.release() 
cv2.destroyAllWindows()

: 